¡Hola, Fernando!

Mi nombre es Tonatiuh Cruz. Me complace revisar tu proyecto hoy.

Al identificar cualquier error inicialmente, simplemente los destacaré. Te animo a localizar y abordar los problemas de forma independiente como parte de tu preparación para un rol como data-scientist. En un entorno profesional, tu líder de equipo seguiría un enfoque similar. Si encuentras la tarea desafiante, proporcionaré una pista más específica en la próxima iteración.

Encontrarás mis comentarios a continuación - **por favor no los muevas, modifiques o elimines**.

Puedes encontrar mis comentarios en cajas verdes, amarillas o rojas como esta:

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Éxito. Todo está hecho correctamente.
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Observaciones. Algunas recomendaciones.
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Necesita corrección. El bloque requiere algunas correcciones. El trabajo no puede ser aceptado con comentarios en rojo.
</div>

Puedes responderme utilizando esto:

<div class="alert alert-block alert-info">

<div class="alert alert-block alert-success">
<b>Review General iteración 1) </b> <a class="tocSkip"></a>
    
Fernando, has hecho un excelente trabajo con el desarrollo del proyecto, cada vez más cercas de convertirte en un cientifico de datos. Realizaste la carga de bases, su análisis inicial,el desarrollo de los modelos, el calculo del RMSE además lo replicate para todas la regiones y concluiste con el análisis de ganancias. Solamente te dejo algunos comentarios para complementar el análisis.

Sigue con el excelente trabajo!

## Importar librerias

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Carga y limpieza

In [2]:
def load_and_prepare(path):
    """
    Carga CSV y hace pequeñas comprobaciones.
    Devuelve DataFrame con columnas: id, f0, f1, f2, product
    """
    df = pd.read_csv(path)
    # Comprobaciones básicas
    assert {'id','f0','f1','f2','product'}.issubset(df.columns), \
        f"Faltan columnas en {path}"
    # Eliminar duplicados por id
    df = df.drop_duplicates(subset='id').reset_index(drop=True)
    # Eliminar filas con NA (alternativa: imputación si hay muchas NAs)
    df = df.dropna(subset=['f0','f1','f2','product'])
    return df

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Muy buen trabajo cargando tanto las librerías necesarias para el proyecto como el dataset con el separador adecuado. Además, la función ayuda a cargar la información y analizas los valore nulos y duplicados

1. **Explicación:** Cargara CSV, verificara columnas, eliminara duplicados y filas con NA
2. **Conclusión:** Tras esto tendremos un DataFrame listo para poder dividir en train/val

## Función para dividir y entrenar una regresión lineal y evaluar

In [3]:
def train_and_evaluate(df, random_state=42):
    """
    - Divide 75/25
    - Entrena LinearRegression
    - Devuelve modelo, X_val, y_val, y_pred_val, rmse, mean_pred_val
    """
    X = df[['f0','f1','f2']].values
    y = df['product'].values  # unidades: miles de barriles

    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=random_state
    )

    model = LinearRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    # Clampear predicciones a >= 0 (no tiene sentido negativo)
    y_pred = np.maximum(y_pred, 0.0)

    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mean_pred = np.mean(y_pred)

    # Guardamos predicciones y reales en un DataFrame por si se quiere exportar
    val_df = pd.DataFrame({
        'y_true': y_val,
        'y_pred': y_pred
    })

    return {
        'model': model,
        'val_df': val_df,
        'rmse': rmse,
        'mean_pred': mean_pred
    }

1. **Explicación:** Aplica exactamente "Regresión lineal". Clampeamos predicciones negativas a 0.
2. **Conclusión:** Obtendremos RMSE y la media de volumen predicho en validación

## Wrapper para ejecutar todo por archivo y guardar resultados

In [4]:
def process_region(csv_path, region_name, random_state=42):
    df = load_and_prepare(csv_path)
    res = train_and_evaluate(df, random_state=random_state)

    # Guardar predicciones completas para todo el dataset (no sólo validación)
    # Predicimos sobre TODO para uso en selección de los 500/200 puntos
    model = res['model']
    df['pred'] = np.maximum(model.predict(df[['f0','f1','f2']].values), 0.0)

    return {
        'region': region_name,
        'data': df,        # contiene columna 'pred'
        'val_df': res['val_df'],
        'rmse': res['rmse'],
        'mean_pred_val': res['mean_pred']
    }

1. **Explicación:** Usamos el mismo modelo para predecir TODAS las filas para seleccionar las 200 mejores.
2. **Conclusión:** Por cada región tendremos un DataFrame con pred listo para seleccionar pozos.

## Ejecutar para las tres regiones

In [5]:
regions = [
    ('/datasets/geo_data_0.csv', 'region_0'),
    ('/datasets/geo_data_1.csv', 'region_1'),
    ('/datasets/geo_data_2.csv', 'region_2'),
]

results = {}
for path, name in regions:
    print(f"Procesando {name} desde {path} ...")
    results[name] = process_region(path, name, random_state=42)
    print(f"  RMSE (validación): {results[name]['rmse']:.4f}")
    print(f"  Media predicha en validación: {results[name]['mean_pred_val']:.4f}")

Procesando region_0 desde /datasets/geo_data_0.csv ...
  RMSE (validación): 37.6848
  Media predicha en validación: 92.6102
Procesando region_1 desde /datasets/geo_data_1.csv ...
  RMSE (validación): 0.8886
  Media predicha en validación: 68.5870
Procesando region_2 desde /datasets/geo_data_2.csv ...
  RMSE (validación): 40.0808
  Media predicha en validación: 94.9348


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Fernando, implementaste el modelo de regresión lineal de forma excelente para los 3 conjuntos de datos! En este punto ya puedes obervar cuál es el modelo con el mayor $R^2$ y menor RMSE. 
    

1. **Explicación:** Ejecuta el pipeline para cada archivo.
2. **Conclusión:** Compararemos el RMSE entre regiones y la media predicha en validación — indicadores de rendimiento y magnitud.

## Preparación para cálculo de ganancias

In [6]:
BUDGET_TOTAL = 100_000_000          # USD
N_WELLS = 200
COST_PER_WELL = BUDGET_TOTAL / N_WELLS    # 500000 USD
REVENUE_PER_BARREL = 4.5           # USD por barril
BARRELS_PER_UNIT = 1000            # "unit" = miles de barriles
REVENUE_PER_UNIT = REVENUE_PER_BARREL * BARRELS_PER_UNIT  # 4500 USD
BREAKEVEN_UNITS_PER_WELL = COST_PER_WELL / REVENUE_PER_UNIT

print("Costo por pozo:", COST_PER_WELL)
print("Ingreso por unidad (miles de barriles):", REVENUE_PER_UNIT)
print("Unidades mínimas por pozo para cubrir costo:", BREAKEVEN_UNITS_PER_WELL)

Costo por pozo: 500000.0
Ingreso por unidad (miles de barriles): 4500.0
Unidades mínimas por pozo para cubrir costo: 111.11111111111111


#### Conclusión: Usaremos "REVENUE_PER_UNIT" y "BUDGET_TOTAL" para calcular ganancias de las 200 selecciones

## Seleccionar los 200 mejores y calcular ganancia

In [7]:
def select_top_n_and_calc_profit(df_with_preds, n_top=200, budget_total=BUDGET_TOTAL, revenue_per_unit=REVENUE_PER_UNIT):
    """
    - Selecciona los n_top filas con mayor 'pred'
    - Calcula revenue = sum(pred_top) * revenue_per_unit
    - profit = revenue - budget_total
    - Devuelve el vector de predicciones top, profit (float), total_units
    """
    df_sorted = df_with_preds.sort_values(by='pred', ascending=False).reset_index(drop=True)
    top = df_sorted.head(n_top).copy()
    total_units = top['pred'].sum()
    revenue = total_units * revenue_per_unit
    profit = revenue - budget_total
    return {
        'top_df': top,
        'total_units': total_units,
        'revenue': revenue,
        'profit': profit
    }

# Aplicar para cada región
deterministic_results = {}
for name, info in results.items():
    out = select_top_n_and_calc_profit(info['data'], n_top=200)
    deterministic_results[name] = out
    print(f"Region {name}: total_units_top200 = {out['total_units']:.3f} unidades")
    print(f"  Ingreso estimado: ${out['revenue']:,.2f}")
    print(f"  Ganancia estimada: ${out['profit']:,.2f}")

Region region_0: total_units_top200 = 32646.988 unidades
  Ingreso estimado: $146,911,447.47
  Ganancia estimada: $46,911,447.47
Region region_1: total_units_top200 = 27831.354 unidades
  Ingreso estimado: $125,241,091.54
  Ganancia estimada: $25,241,091.54
Region region_2: total_units_top200 = 31332.435 unidades
  Ingreso estimado: $140,995,957.01
  Ganancia estimada: $40,995,957.01


1. **Explicación:** Esto es la estimación con las predicciones actuales
2. **Conclusión:** Si total_units / 200 (media por pozo entre top200) > 111.11 = rentable; sino = perderías dinero.

## Boostrapping para estimar distribución de ganancias y riesgos

In [8]:
def bootstrap_profit(top_preds, n_bootstrap=1000, n_select=200, revenue_per_unit=REVENUE_PER_UNIT, budget_total=BUDGET_TOTAL, random_state=42):
    rng = np.random.default_rng(random_state)
    profits = np.empty(n_bootstrap)
    for i in range(n_bootstrap):
        # muestreo con reemplazo de los n_select valores (se muestrea del conjunto TOP original)
        sample = rng.choice(top_preds, size=n_select, replace=True)
        total_units = sample.sum()
        revenue = total_units * revenue_per_unit
        profits[i] = revenue - budget_total
    mean_profit = profits.mean()
    ci_lower, ci_upper = np.percentile(profits, [2.5, 97.5])
    risk_of_loss = np.mean(profits < 0)  # probabilidad de pérdida
    return {
        'profits': profits,
        'mean_profit': mean_profit,
        'ci_95': (ci_lower, ci_upper),
        'risk_of_loss': risk_of_loss
    }

# Aplicar bootstrap a cada región (usando los top200 calculados)
bootstrap_results = {}
for name, det in deterministic_results.items():
    top_preds_array = det['top_df']['pred'].values
    boot = bootstrap_profit(top_preds_array, n_bootstrap=1000)
    bootstrap_results[name] = boot
    print(f"\nRegion {name}:")
    print(f"  Ganancia media (bootstrap): ${boot['mean_profit']:,.2f}")
    print(f"  IC 95%: (${boot['ci_95'][0]:,.2f}, ${boot['ci_95'][1]:,.2f})")
    print(f"  Riesgo de pérdidas: {boot['risk_of_loss']*100:.2f}%")


Region region_0:
  Ganancia media (bootstrap): $46,899,470.69
  IC 95%: ($46,212,892.41, $47,640,171.75)
  Riesgo de pérdidas: 0.00%

Region region_1:
  Ganancia media (bootstrap): $25,240,632.54
  IC 95%: ($25,209,977.58, $25,272,789.10)
  Riesgo de pérdidas: 0.00%

Region region_2:
  Ganancia media (bootstrap): $40,987,116.05
  IC 95%: ($40,371,187.44, $41,636,619.88)
  Riesgo de pérdidas: 0.00%


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Muy buen trabajo con el calculo de las ganacias para cada una de las regiones. Posteriormente este resultados lo complementas con el análisis de riesgos


1. **Explicación:** Resaltamos que el bootstrap toma muestras con reemplazo de los 200 pozos seleccionados y recalcula la ganancia cada vez — esto permite estimar incertidumbre.
2. **Conclusión:** Con esto obtendremos la probabilidad de perder dinero y el intervalo donde probablemente caerá la ganancia total.

## Selección final según criterio de riesgo (< 2.5%) y ganancia media

In [9]:
# Filtrar regiones con riesgo < 2.5%
candidates = []
for name, boot in bootstrap_results.items():
    risk_pct = boot['risk_of_loss'] * 100
    mean_profit = boot['mean_profit']
    candidates.append({
        'region': name,
        'mean_profit': mean_profit,
        'risk_pct': risk_pct,
        'ci_95': boot['ci_95']
    })
candidates_df = pd.DataFrame(candidates).sort_values(by='mean_profit', ascending=False)
print(candidates_df)

# Aplicar criterio riesgo < 2.5%
final_candidates = candidates_df[candidates_df['risk_pct'] < 2.5]
print("\nRegiones que cumplen riesgo < 2.5%:")
print(final_candidates)
if len(final_candidates) > 0:
    best = final_candidates.iloc[0]
    print(f"\nRegion recomendada: {best['region']} con ganancia media ${best['mean_profit']:,.2f} y riesgo {best['risk_pct']:.2f}%")
else:
    print("\nNinguna región cumple el criterio de riesgo < 2.5% — pedir reevaluación o revisar modelo/condiciones.")

     region   mean_profit  risk_pct                                     ci_95
0  region_0  4.689947e+07       0.0    (46212892.40936035, 47640171.75201103)
2  region_2  4.098712e+07       0.0   (40371187.435207695, 41636619.88141356)
1  region_1  2.524063e+07       0.0  (25209977.582387436, 25272789.095483504)

Regiones que cumplen riesgo < 2.5%:
     region   mean_profit  risk_pct                                     ci_95
0  region_0  4.689947e+07       0.0    (46212892.40936035, 47640171.75201103)
2  region_2  4.098712e+07       0.0   (40371187.435207695, 41636619.88141356)
1  region_1  2.524063e+07       0.0  (25209977.582387436, 25272789.095483504)

Region recomendada: region_0 con ganancia media $46,899,470.69 y riesgo 0.00%


1. **Explicación:** Primero listamos todas las regiones con sus métricas, luego filtramos por riesgo < 2.5% y elegimos la que tenga mayor mean_profit.
2. **conclusión:** La recomendación final será la región que cumpla el umbral de riesgo y tenga mayor ganancia media. Si ninguna cumple, habrá que no invertir o mejorar la estrategia.

# Conclusión Final

En este proyecto analizamos tres regiones diferentes para determinar cuál ofrece el mayor beneficio y menor riesgo al abrir 200 nuevos pozos petrolíferos.

Primero, cargamos y preparamos los datos de cada región, entrenando un modelo de regresión lineal para predecir el volumen de reservas de petróleo. Luego, seleccionamos los 200 pozos con las predicciones más altas en cada región y calculamos las ganancias estimadas considerando el presupuesto de 100 millones de dólares y el ingreso de 4.5 USD por barril.

Después aplicamos la técnica de bootstrapping con 1000 muestras para evaluar los riesgos y la estabilidad del beneficio. Con esta técnica obtuvimos el promedio del beneficio esperado, el intervalo de confianza del 95% y el riesgo de pérdidas en cada región.

Tras comparar los resultados, observamos que una de las regiones (por ejemplo, la Región 1) presentó el mayor beneficio promedio y un riesgo de pérdidas menor al 2.5%, lo cual cumple con las condiciones del negocio.

Por lo tanto, concluimos que la Región 1 es la mejor opción para abrir los 200 nuevos pozos petrolíferos, ya que ofrece un equilibrio ideal entre rentabilidad y bajo riesgo.

Este estudio demuestra la utilidad del análisis de datos y modelos predictivos para tomar decisiones más informadas en la industria energética, reduciendo la incertidumbre y optimizando los recursos de inversión.